In [ ]:
# keras module for building LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

# set seeds for reproducability
import tensorflow as tf
from numpy.random import seed
seed(1)

import pandas as pd
import numpy as np
import string, os

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/summer internship(Gen Ai)/assignment 3/ArticlesApril2017.csv')
df.head()


In [ ]:
curr_dir = '/content/drive/MyDrive/summer internship(Gen Ai)/assignment 3/'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
        break

all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

In [ ]:
print(all_headlines[:5])

In [ ]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [ ]:
tokenizer.word_index

In [ ]:
len(tokenizer.word_index)

In [ ]:
for sentence in corpus[:10]:
    print(sentence,"\n",tokenizer.texts_to_sequences([sentence])[0])

In [ ]:
x_train =[]
label=[]
for sentence in corpus:
    toknized_sentence=tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1,len(toknized_sentence)):
        n_gram_sequence=toknized_sentence[:i+1]
        x_train.append(n_gram_sequence)
        label.append(toknized_sentence[i])

In [ ]:
x_train[:10]

In [ ]:
label[:10]

In [ ]:
for i,j in zip(x_train[:50],label[:50]):
    print(i,j)

In [ ]:
maxlen=max([len(i) for i in x_train])
# pad_sequences(x_train,maxlen=maxlen,padding='pre')
x_train=pad_sequences(x_train,maxlen=10,padding='pre')
label=np.array(label)
print(x_train)
print(label)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1,output_dim=100,input_length=max([len(i) for i in x_train])))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(len(tokenizer.word_index)+1,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train,label,epochs=30,verbose=1)

Epoch 1/30
151/151 [==============================] - 6s 43ms/step - loss: 3.8552 - accuracy: 0.2640
Epoch 2/30
151/151 [==============================] - 4s 27ms/step - loss: 3.6913 - accuracy: 0.3007
Epoch 3/30
151/151 [==============================] - 4s 27ms/step - loss: 3.5442 - accuracy: 0.3325
Epoch 4/30
151/151 [==============================] - 7s 43ms/step - loss: 3.3874 - accuracy: 0.3606
Epoch 5/30
151/151 [==============================] - 4s 27ms/step - loss: 3.2327 - accuracy: 0.3951
Epoch 6/30
151/151 [==============================] - 4s 28ms/step - loss: 3.0867 - accuracy: 0.4263
Epoch 7/30
151/151 [==============================] - 6s 43ms/step - loss: 2.9412 - accuracy: 0.4551
Epoch 8/30
151/151 [==============================] - 4s 27ms/step - loss: 2.8029 - accuracy: 0.4873
Epoch 9/30
151/151 [==============================] - 4s 27ms/step - loss: 2.6668 - accuracy: 0.5162
Epoch 10/30
151/151 [==============================] - 6s 40ms/step - loss: 2.5255 - accura

In [ ]:
# def generate_text(seed_text, next_words, model, max_sequence_len):
#     for _ in range(next_words):
#         token_list = tokenizer.texts_to_sequences([seed_text])[0]
#         token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
#         predicted = model.predict(token_list)
#         # predictions = np.argmax(model.predict(token_list), axis=-1)
#         output_word = ""
#         for word,index in tokenizer.word_index.items():
#             if index == predicted:
#                 output_word = word
#                 break
#         seed_text += " "+output_word
#     return seed_text.title()
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)

        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
print(generate_text('view of a' , 4 , model , maxlen))